In [0]:
import urllib.request
import json
import pandas as pd
import textwrap
from bs4 import BeautifulSoup as soup
import requests
import gspread

In [0]:
Isbnlist=[9780143064978,9781447209225,9788190105972,9780330302296,9780143330219,9788174364166,9788189988364,9788120781757,9780143441502,9788129300379,9781420070644,9789386322265,9788181479136]

In [0]:
for i in range(len(Isbnlist)):
  y=Isbnlist[i]
  Isbnlist[i]=str(y)

In [0]:
def getBookValsGoogleBooks(isbn="9788129300379"):

  base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

  savelist=["publishedDate","title","authors","pageCount","categories","infoLink"]
  namedik={'Author': 'Swami Ramsukhdas','Categories': 'Conduct of life','ISBN-13': '9788129300379','Published': '2010','Title': 'Be Good and Invaluable Advice','URL': 'http://books.google.com/books?id=zOUmMwEACAAJ&dq=isbn:9788129300379&hl=&source=gbs_api'}

  with urllib.request.urlopen(base_api_link + isbn) as f:
    text = f.read()
    decoded_text = text.decode("utf-8")
    obj = json.loads(decoded_text) 

    if obj["totalItems"] <= 0:
      errdict={}
      for i in namedik.keys():
        errdict[i]=""
      return errdict
        
    
    volume_info = obj["items"][0] 
    for i in savelist:
      if i not in volume_info['volumeInfo'].keys():
        volume_info['volumeInfo'][i]=''

    authors = obj["items"][0]["volumeInfo"]["authors"]

  df=pd.DataFrame.from_dict(volume_info)
  df.drop(columns=['kind','id','etag','selfLink','saleInfo','accessInfo'],inplace=True)
  df=df.transpose()
  labeltodroplist=[]

  authlist=df['authors'][0]
  df['authors'][0]=", ".join(authlist)

  if df['categories'] is not '':
    catlist=df['categories'][0]
    df['categories'][0]=", ".join(catlist)
  
  for i in range(len(df.transpose())):
    if df.columns[i] not in savelist:
      labeltodroplist.append(df.columns[i])
  df.drop(inplace=True,axis='columns',labels=labeltodroplist)
  df.rename(columns={"authors":"Author","categories":"Categories","publishedDate":"Published","title":"Title","infoLink":"URL"},inplace=True)
  #df['ISBN-13']=[(isbn)]
  df.drop(inplace=True,columns=["pageCount"])
  print(df.to_dict(orient='records')[0])
  return(df.to_dict(orient='records')[0])


In [0]:
def getpageISBN(isbn="9788129300379"):

  apikey="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  apiurl="http://isbnsearch.org/isbn/"

  #r=requests.get(apiurl+isbn)
  #if r is "<Response [403]>":
  req=requests.get(apikey+apiurl+isbn)
  return(req)

def getbookvals(r):
  page_soup=soup(r.content,"html.parser")
  #print(page_soup.prettify())
  booklist=page_soup.findAll("div",{"class":"bookinfo"})
  if len(booklist)==0:
      return {"ISBN-13":isbn}
  page2=soup(str(booklist[0]),"html.parser")
  title=soup((str(page2.find("h1"))),"html.parser").text
  plist=page2.findAll("p")
  infol=[('Title',title)]
  for i in plist:
    vals=i.text.split(sep=": ",maxsplit=1)
    infol.append((vals[0],vals[1]))
  infodict=dict(infol)
  infodict["URL"]=str("http://isbnsearch.org/isbn/"+infodict["ISBN-13"])
  return(infodict)

In [0]:
infolist=[]
row={}
gRowdict={}
for i in Isbnlist:
  gRowdict=getBookValsGoogleBooks(i)
  gRowdict["ISBN-13"]=i
  infolist.append(gRowdict)

{'Title': 'The Penguin Yearbook 2009', 'Author': "Derek O'Brien", 'Published': '2008', 'Categories': 'Encyclopedias and dictionaries', 'URL': 'http://books.google.co.in/books?id=2khyPwAACAAJ&dq=isbn:9780143064978&hl=&source=gbs_api'}
{'Title': 'Effective Time Management', 'Author': 'John Eric Adair', 'Published': '2002-05-01', 'Categories': 'Reference', 'URL': 'http://books.google.co.in/books?id=DairQgAACAAJ&dq=isbn:9780330302296&hl=&source=gbs_api'}
{'Title': 'Boyhood Days', 'Author': 'Rabindranath Tagore', 'Published': '2007', 'Categories': 'Biography & Autobiography', 'URL': 'http://books.google.co.in/books?id=UWhnwlQyplEC&dq=isbn:9780143330219&hl=&source=gbs_api'}
{'Title': 'Nutitive Cooking For Children', 'Author': 'Dr Shaila Santhanam', 'Published': '2006-01-01', 'URL': 'http://books.google.co.in/books?id=PzeSMwAACAAJ&dq=isbn:9788174364166&hl=&source=gbs_api', 'Categories': ''}
{'Title': 'Jagat Guru', 'Author': 'Prasada Jagannadha Rao', 'Published': '2013-01-01', 'Categories': 'S

In [0]:
for i in range(len(infolist)):
  y=infolist[i]
  if (y["Author"] is "" and y["Title"] is ""):
    #not found in google books.
    #find in the web scraper method
    isbn=Isbnlist[i]
    rst=getpageISBN(isbn)
    l1=getbookvals(rst)
    infolist[i]=l1

In [0]:
df=pd.DataFrame(infolist)
df

,Title,Author,Published,Categories,URL,ISBN-13,ISBN-10,Edition,Binding,Publisher
0,The Penguin Yearbook 2009,Derek O'Brien,2008,Encyclopedias and dictionaries,http://books.google.co.in/books?id=2khyPwAACAA...,9780143064978,NaN,NaN,NaN,NaN
1,The Sins of the Father (The Clifton Chronicles),Jeffrey Archer,January 2012,NaN,http://isbnsearch.org/isbn/9781447209225,9781447209225,1447209222,Indian ed,Paperback,Pan MacMillan
2,NaN,NaN,NaN,NaN,NaN,9788190105972,NaN,NaN,NaN,NaN
3,Effective Time Management,John Eric Adair,2002-05-01,Reference,http://books.google.co.in/books?id=DairQgAACAA...,9780330302296,NaN,NaN,NaN,NaN
4,Boyhood Days,Rabindranath Tagore,2007,Biography & Autobiography,http://books.google.co.in/books?id=UWhnwlQyplE...,9780143330219,NaN,NaN,NaN,NaN
5,Nutitive Cooking For Children,Dr Shaila Santhanam,2006-01-01,,http://books.google.co.in/books?id=PzeSMwAACAA...,9788174364166,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,9788189988364,NaN,NaN,NaN,NaN
7,Jagat Guru,Prasada Jagannadha Rao,2013-01-01,Spiritual life,http://books.google.co.in/books?id=5EaPngEACAA...,9788120781757,NaN,NaN,NaN,NaN
8,Exam Warriors,"Modi, Narendra",January 2019,NaN,http://isbnsearch.org/isbn/9780143441502,9780143441502,0143441507,NaN,Paperback,Penguin Random House India
9,Be Good and Invaluable Advice,Swami Ramsukhdas,2010,Conduct of life,http://books.google.co.in/books?id=zOUmMwEACAA...,9788129300379,NaN,NaN,NaN,NaN


In [0]:
df.drop(inplace=True,columns=["ISBN-10","Edition","Binding","Publisher"])
df.fillna("Not Available")

,Title,Author,Published,Categories,URL,ISBN-13
0,The Penguin Yearbook 2009,Derek O'Brien,2008,Encyclopedias and dictionaries,http://books.google.co.in/books?id=2khyPwAACAA...,9780143064978
1,The Sins of the Father (The Clifton Chronicles),Jeffrey Archer,January 2012,Not Available,http://isbnsearch.org/isbn/9781447209225,9781447209225
2,Not Available,Not Available,Not Available,Not Available,Not Available,9788190105972
3,Effective Time Management,John Eric Adair,2002-05-01,Reference,http://books.google.co.in/books?id=DairQgAACAA...,9780330302296
4,Boyhood Days,Rabindranath Tagore,2007,Biography & Autobiography,http://books.google.co.in/books?id=UWhnwlQyplE...,9780143330219
5,Nutitive Cooking For Children,Dr Shaila Santhanam,2006-01-01,,http://books.google.co.in/books?id=PzeSMwAACAA...,9788174364166
6,Not Available,Not Available,Not Available,Not Available,Not Available,9788189988364
7,Jagat Guru,Prasada Jagannadha Rao,2013-01-01,Spiritual life,http://books.google.co.in/books?id=5EaPngEACAA...,9788120781757
8,Exam Warriors,"Modi, Narendra",January 2019,Not Available,http://isbnsearch.org/isbn/9780143441502,9780143441502
9,Be Good and Invaluable Advice,Swami Ramsukhdas,2010,Conduct of life,http://books.google.co.in/books?id=zOUmMwEACAA...,9788129300379


In [0]:
df.to_csv("./data.csv",index=False)